In [1]:
using JuMP
using CPLEX
using LinearAlgebra

Now we define the master problem :)

In [2]:
function solve_master(A,b,c, op_A, op_B, feas_A, feas_B)
    nx = size(c, 2)
    m = Model(CPLEX.Optimizer)
    set_optimizer_attribute(m, "CPX_PARAM_SCRIND", 0)
    if size(op_A, 1) == 0
        
        @variable(m, x[i=1:nx]>=0)
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx))
        println("Master problem is : ")
        print("\tMin ")
        println(sum(c[i] * x[i] for i = 1:nx))
        println("\tsuch that ")
        print("\t\t")
        print(A*x)
        print("<= ")
        println(b)
        @constraint(m, A*x .<= b)
        theta = -Inf
    else
        @variable(m, x[1:nx])
        @variable(m, theta)
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx) + theta)
        @constraint(m, A*x .<= b)
        println("Master problem is : ")
        print("\tMin ")
        println(sum(c[i] * x[i] for i = 1:nx) + theta)
        println("\tsuch that ")
        print("\t\t")
        print(A*x)
        print("<= ")
        println(b)
        for i = 1:size(op_A, 1)
            @constraint(m, sum(op_A[i,j] * x[j] for j = 1:nx) + op_A[i,nx+1] * theta .>= op_B[i])
            print("\t\t")
            print(sum(op_A[i,j] * x[j] for j = 1:nx) + op_A[i,nx+1] * theta)
            print(".>= ")
            println(op_B[i])
    
        end
        
        
    end
    if size(feas_A, 1) != 0
        for i = 1:size(feas_A, 1)
            @constraint(m, sum(feas_A[i,j] * x[j] for j = 1:nx) .>= feas_B[i])
            print("\t\t")
            print(sum(feas_A[i,j] * x[j] for j = 1:nx))
            print(".>= ")
            println(feas_B[i])
        end
    end
    for i = 1:nx
        print("\t\t")
        print(x[i])
        print(">=")
        println(0)
    end
    optimize!(m)
    theta = JuMP.value(theta)
    sol = zeros(1, nx)
    for i = 1:nx
        sol[i] = JuMP.value(x[i])
    end
    sol = vec(sol)
    print("\tSolution : \n")
    print("\t\tx = ")
    println(sol)
    print("\t\ttheta = ")
    println(theta)
    println()
    return (objective_value(m), sol, theta, JuMP.termination_status(m))
end

solve_master (generic function with 1 method)

In [3]:
function step2(W, h, T, x, omega, q)
    m = Model(CPLEX.Optimizer)
    ny = size(W,2)
    set_optimizer_attribute(m, "CPX_PARAM_SCRIND", 0)
    @variable(m, y[1:ny] >= 0)
    @constraint(m,constraint, W*y - h[:,omega] + T*x .<= 0)
    @objective(m, Min, transpose(q[omega,:]) * y)
    print("\t\tMin")
    println(transpose(q[omega,:]) * y)
    println("\t\tSuch that :")
    print("\t\t\t")
    print(W*y - h[:,omega] + T*x)
    println(".<= 0")
    for i in 1:ny
        print("\t\t\t")
        print(y[i])
        println(">= 0")
    end
    optimize!(m);
    sol = zeros(1, ny)
    for i = 1:ny
        sol[i] = JuMP.value(y[i])
    end
    sol = vec(sol)
    println("\t\tSolution :")
    print("\t\t\ty = ")
    println(sol)
    print("\t\t\tdual = ")
    println(dual.(constraint))
    return (objective_value(m), sol, dual.(constraint), JuMP.termination_status(m))
end

step2 (generic function with 1 method)

In [4]:
function compute_optimality_cuts(W, h, T, x_k, q, p)
    println("Computation of optimality cuts : ")
    e = 0
    E = zeros(1,size(x_k,1))
    w_now = 0
    y_now = [] 
    for w in 1:size(p,2)
        print("\tProblem : ")
        println(w)
        (w_now, y_now, pi_k, termination_status) = step2(W, h, T, x_k, w, q)
        e += p[w] * transpose(pi_k) * h[:,w]
        E += p[w] * transpose(pi_k) * T
    end
    return(e, E, e - (E*x_k)[1], y_now)
end

compute_optimality_cuts (generic function with 1 method)

In [9]:
function compute_feasibility_cuts(W,h,T,x_k)
    ny = size(W,2)        # number of variable
    nconst = size(W,1)    # number of constraint
    nscen = size(h,2)     # number of scenario
    println("Computation of feasibility cuts : ")
    for k in 1:nscen
        print("\tProblem : ")
        println(k)
        m = Model(CPLEX.Optimizer)
        set_optimizer_attribute(m, "CPX_PARAM_SCRIND", 0)
        @variable(m, y[1:ny] >= 0)
        @variable(m, vplus[1:nconst] >= 0)
        @variable(m, vmoins[1:nconst] >= 0)
        @objective(m, Min, sum(vplus[i] + vmoins[i] for i = 1:nconst))
        @constraint(m, dual_const, W*y + vplus - vmoins .<= h[:,k] - T*x_k) # .<= instead of .== because we add one variable for each equality : x + y = 10 <=> x + y - s <= 10
                                                                            # Dual shows that indead the constraint is tight when w' = 0
        print("\t\tMin")
        println(transpose(sum(vplus[i] + vmoins[i] for i = 1:nconst)))
        println("\t\tSuch that :")
        print("\t\t\t")
        print(W*y + vplus - vmoins)
        print(".<= ")
        println(h[:,k] - T*x_k)
        for i in 1:ny
            print("\t\t\t")
            print(y[i])
            println(">= 0")
        end
        for i in 1:nconst
            print("\t\t\t")
            print(vplus[i])
            println(">= 0")
        end 
        for i in 1:nconst
            print("\t\t\t")
            print(vmoins[i])
            println(">= 0")
        end
        optimize!(m)
        w_prime = objective_value(m)
        dual_val = dual.(dual_const)
        println("\tSolution:")
        print("\t\tw' = ")
        println(w_prime)
        print("\t\tdual = ")
        println(dual_val)
        println()
        if(w_prime > 0)
            D = transpose(dual_val) * T
            d = transpose(dual_val) * h[:,k]
            return(true,d,D)
        end
    end
    return(false,-1,-1)
end

compute_feasibility_cuts (generic function with 1 method)

In [10]:
function main(A, b, c, T, W, h, q, p)
    op_A = []
    op_B = []
    fea_A = []
    fea_B = []
    k = 0
    val_k, x_k, theta_k = 0, 0, 0
    w_now = -Inf
    y_k = 0
    e = 0
    E = zeros(1,size(p,2))
    while (k<10000000000)
        print("Iteration : ")
        println(k+1)
        (val_k, x_k, theta_k, termination_status) = solve_master(A, b, c, op_A, op_B, fea_A, fea_B)
        if termination_status == MOI.INFEASIBLE
            return (-1, -1, -1, -1)
        end
        V_temp = []
        (added_const,d,D) = compute_feasibility_cuts(W,h,T,x_k)
        println()
        if !added_const
            (e, E, w_now, y_k) = compute_optimality_cuts(W, h, T, x_k, q, p)
            print("\tw' = ")
            println(w_now)
            print("\tAdded optimality constraint : ")
            for i in 1:size(x_k,1)
                print(E[i])
                print(" * x[")
                print(i)
                print("]")
            end
            print(" + theta >= ")
            println(e)
            if(w_now <= theta_k)
                return (x_k, y_k, k+1, val_k)
            end
            V_temp = [E 1]
            if op_A == []
                op_A  = V_temp
                op_B  = [e]    
            else
                op_A = [op_A; V_temp]
                op_B = [op_B;e]
            end
        else
            print("\tAdded feasibility constraint : ")
            for i in 1:size(x_k,1)
                print(D[i])
                print(" * x[")
                print(i)
                print("]")
            end
            print(" >= ")
            println(d)
            V_temp = D
            if fea_A == []
                fea_A  = V_temp
                fea_B  = [d]    
            else
                fea_A = [fea_A; V_temp]
                fea_B = [fea_B;d]
            end
        end
        k += 1 
        println()
    end
end 

main (generic function with 1 method)

In [12]:

# Test 1
d1 = [500 100; 300 300]
q1 = [-24 -28; -28 -32]
p1 = [0.4 0.6]

A1 = [1 1; -1 0; 0 -1]
b1 = [120; -40; -20]
c1 = [100 150]

T1 = [-60 0; 0 -80; 0 0; 0 0]
W1 = [6 10; 8 5; 1 0; 0 1]
h1 = [0 0; 0 0; d1[1,1] d1[2,1]; d1[1,2] d1[2,2]]
#
# Test 2

q2 = [1 1; 1 1; 1 1]
p2 = [1/3 1/3 1/3]
A2 = reshape([1], 1, 1)
b2 = [10]
c2 = [0]
T2 = reshape([1; -1], 2, 1)
W2 = [1 -1; -1 1]
h2 = [1 2 4; -1 -2 -4]


# Test 3

q3 = [15 12; 15 12; 15 12; 15 12]
p3 = [1/4 1/4 1/4 1/4]
A3 = reshape([0 0], 1, 2)
b3 = [1]
c3 = [3 2]
T3 = [-1 0; 0 -1; 0 0; 0 0; 0 0; 0 0]
W3 = [3 2; 2 5; 1 0; 0 1; -1 0; 0 -1]
#h3 = [0 0 0 0; 0 0 0 0; 4 4 6 6; 4 8 4 8; (-0.8*4) (-0.8*4) (-0.8*6) (-0.8*6); (-0.8*4) (-0.8*8) (-0.8*4) (-0.8*8)]
h3 = [0 0 0 0; 0 0 0 0; 6 4 6 4; 8 8 4 4; (-0.8*6) (-0.8*4) (-0.8*6) (-0.8*4); (-0.8*8) (-0.8*8) (-0.8*4) (-0.8*4)]
#
print("\n\n===Solving test 1 ===\n\n")
(x,y, k, val) = main(A1, b1, c1, T1, W1, h1, q1, p1)

println()
println("---------")
println("Solution Found !")
print("\t x = ")
println(x)
print("\t Iterations = ")
println(k)
println("---------")
print("\n\n===Solving test 2 ===\n\n")
(x,y, k, val) = main(A2, b2, c2, T2, W2, h2, q2, p2)
println()
println("---------")
println("Solution Found !")
print("\t x = ")
println(x)
print("\t Iterations = ")
println(k)
println("---------")
print("\n\n===Solving test 3 ===\n\n")
(x,y, k, val) = main(A3, b3, c3, T3, W3, h3, q3, p3)
println()
println("---------")
println("Solution Found !")
print("\t x = ")
println(x)
print("\t Iterations = ")
println(k)
println("---------")
println("This a part of a text to test if the ssh connection to github works well :)")



===Solving test 1 ===

Iteration : 1
Master problem is : 
	Min 100 x[1] + 150 x[2]
	such that 
		AffExpr[x[1] + x[2], -x[1], -x[2]]<= [120, -40, -20]
		x[1]>=0
		x[2]>=0
	Solution : 
		x = [40.0, 20.0]
		theta = -Inf

Computation of feasibility cuts : 
	Problem : 1
		Minvplus[1] + vmoins[1] + vplus[2] + vmoins[2] + vplus[3] + vmoins[3] + vplus[4] + vmoins[4]
		Such that :
			AffExpr[6 y[1] + 10 y[2] + vplus[1] - vmoins[1], 8 y[1] + 5 y[2] + vplus[2] - vmoins[2], y[1] + vplus[3] - vmoins[3], y[2] + vplus[4] - vmoins[4]].<= [2400.0, 1600.0, 500.0, 100.0]
			y[1]>= 0
			y[2]>= 0
			vplus[1]>= 0
			vplus[2]>= 0
			vplus[3]>= 0
			vplus[4]>= 0
			vmoins[1]>= 0
			vmoins[2]>= 0
			vmoins[3]>= 0
			vmoins[4]>= 0
	Solution:
		w' = 0.0
		dual = [0.0, 0.0, 0.0, 0.0]

	Problem : 2
		Minvplus[1] + vmoins[1] + vplus[2] + vmoins[2] + vplus[3] + vmoins[3] + vplus[4] + vmoins[4]
		Such that :
			AffExpr[6 y[1] + 10 y[2] + vplus[1] - vmoins[1], 8 y[1] + 5 y[2] + vplus[2] - vmoins[2], y[1] + vplus[3] 